# Mini-project 04 Background
Wed, Nov 23, 2022 (for mini-project due Nov 30)

## *Python background: more on animation*
The following code provides an example of _one_ way to animate a graph with `FuncAnimation`. The line is given by points calculated with the `np.sin()` function, which is updated inside `update_line()` every time the function is called by `FuncAnimation`. In each frame of the animation, the curve is shifted horizontally by an amount set by the variable num.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

number_of_frames = 100


def update_line(num):
    plt.cla()
    plt.plot(np.sin(data + num / 10.0))


data = np.arange(0, 2 * np.pi, 0.01)
fig = plt.figure()
anim = animation.FuncAnimation(fig,
                               update_line,
                               number_of_frames,
                               interval=30,
                               repeat=False)
anim.save('sin.webm', extra_args=['-vcodec', 'libvpx'])
plt.close()
del anim
HTML('<video controls> <source src="sin.webm"' +
     'type="video/webm"></video>')

This approach works, as long as there aren't too many frames to plot for the animation. The problem is that the function `update_line()` generates a brand new plot (axis, frame, lines, etc...) at every function call. This will make generating the animation take more time and memory than necessary. One way around this is to update only the elements of the plot which need to be updated. The example below shows the same line animation where only the line is updated, while everything else stays the same. In order to do this we have to start using some pieces of a more sophisticated interface to matplotlib. Below we store the `line` object in the variable `line` so that we can later update it. The example below shows how to alter the line in a plot using an instance of a plot:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

number_of_frames = 100


def update_line(num):
    # update the data
    line.set_ydata(np.sin(data + num / 10.0))


data = np.arange(0, 2 * np.pi, 0.01)
fig, ax = plt.subplots()
line, = ax.plot(data, np.sin(data))

anim = animation.FuncAnimation(fig,
                               update_line,
                               number_of_frames,
                               interval=30,
                               repeat=False)
anim.save('sin2.webm', extra_args=['-vcodec', 'libvpx'])
plt.close()
del anim
HTML('<video controls> <source src="sin2.webm" ' +
     'type="video/webm"></video>')

The following example shows how you can use `FuncAnimation` to generate animations which involve calculations in a separate function for each iteration. Similar to the line example above, we will use, and update, an instance of `scatter()`, i.e., a scatter plot. The initial plot is a dot located at `xpos`, `ypos`, and its position is updated using the method `set_offsets` of the scatter plot instance `im`. The function `newypos()` updates the position of the dot, shifting vertically by an amount `dypos`:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

number_of_frames = 49
xpos, ypos = 0.5, 0.
dypos = 1. / 50.


def update_point(num):
    global ypos
    ypos = ypos + dypos
    im.set_offsets((xpos, ypos))


fig, ax = plt.subplots()
plt.xlim(0, 1)
plt.ylim(0, 1)
im = ax.scatter(xpos, ypos)

anim = animation.FuncAnimation(fig,
                               update_point,
                               number_of_frames,
                               interval=30,
                               repeat=False)
plt.close()
anim.save('scatter.webm', extra_args=['-vcodec', 'libvpx'])
plt.close()
del anim
HTML('<video controls> <source src="scatter.webm" ' +
     'type="video/webm"></video>')

The last example is similar, but now animates a scatter plot with many points. It uses some additional functions (`numpy.vstack`) and methods of a scatter plot instance (e.g. `set_sizes()` and `set_facecolor()`).

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

npoint = 30
nframe = 100
dstep = 400.


def update_point(num):
    newx = x + np.random.randn(npoint) / dstep
    newy = y + np.random.randn(npoint) / dstep
    data = np.stack((newx, newy), axis=-1)
    im.set_offsets(data)


x = np.random.random(npoint)
y = np.random.random(npoint)
s = 500 * np.random.random(npoint)
c = np.random.random(npoint * 3).reshape(npoint, 3)

fig, ax = plt.subplots()
plt.ylim(-0.2, 1.2)
plt.xlim(-0.2, 1.2)
im = ax.scatter(x, y)
im.set_facecolor(c)
im.set_sizes(s)

anim = animation.FuncAnimation(fig,
                               update_point,
                               nframe,
                               interval=30,
                               repeat=False)
anim.save('scatter2.webm', extra_args=['-vcodec', 'libvpx'])
plt.close()
del anim
HTML('<video controls> <source src="scatter2.webm" ' +
     'type="video/webm"></video>')

## *Python background: `Itertool`*
`itertool` is a powerful python package which can perform all sorts of iterations on array elements. The full description is given at: https://docs.python.org/3/library/itertools.html
 
Here we will only discuss the method `combinations()` which you will need for this project. You will simulate the collision of a large number of particles in a box. In order to decide if two particles collide, you have to test how close they are, and you have to do this for all pairs of particles in the gas. The method `combinations()` can help you do this. Here is an example, where the output is a list of all possible pairings froming from an input list:

In [7]:
import itertools
import numpy as np
x=np.array([1, 2, 3, 4])
list(itertools.combinations(x, 2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

## *Project overview*
The kinetic theory of gases provides the introductory foundations of thermodynamics and statistical physics. A general description is given on the wikipedia page:

* https://en.wikipedia.org/wiki/Kinetic_theory_of_gases

This theory explains the macroscopic properties of gases (e.g., pressure, temperature, entropy) by considering their compositions and particle motions. Of primary importance is the Maxwell-Boltzmann (MB) distribution function which gives the probability distribution function of particle speeds in the gas:

* https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution

This is typically presented as an empirical formula to describe the distributions of speeds in a gas, with little or no discussion of its physical origin. The distribution of speeds is related to the temperature of the gas T is illustrated in the figure:

![](distro.png)

In this project, you will simulate a 2-dimensional gas and recover the MB distribution of velocities. The goal is to develop some understanding as to where the MB distribution comes from microscopically. Your simulation will start with some arbitrary velocity distribution (thus, initially non-MB ) and you will observe that the velocity distribution settles into an MB distribution.

The redistribution of energy in a gas is done through elastic collisions. Every time two particles in the gas collide, you will have to calculate the new velocities according to the conservation of energy and momentum.

There is a simple formula which allows you to do this, (from https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional)

If $v_1$, $v_2$ , $r_1$, and $r_2$ are respectively the velocity and position vectors of particles 1 and 2, then the new velocities after collision will be:

$$\vec{v}_1^\prime = \vec{v}_1 - \frac{(\vec{v}_1-\vec{v}_2)\cdot(\vec{r}_1-\vec{r}_2)}{|\vec{r}_1-\vec{r}_2|^2} (\vec{r}_1-\vec{r}_2),$$
$$\vec{v}_2^\prime = \vec{v}_2 - \frac{(\vec{v}_2-\vec{v}_1)\cdot(\vec{r}_2-\vec{r}_1)}{|\vec{r}_2-\vec{r}_1|^2} (\vec{r}_2-\vec{r}_1),$$

where have assumed that the two particles are circles and have the same mass. This collision configuration is shown in the following diagram:

![](collision.png)

## *Objectives*
You will write a program which follows the collisional motions of particles in a box (see setup instructions below).

Your code should generate several outputs:

**Output 1: Animation.** An animation that shows the particle positions over the time steps (one frame for every 2 steps).

**Output 2: Histograms, fits, and the temperature of your box of gas.**

**2a)** Based on the final state of the simulation you ran for Output 1, create a two-panel plot, with a histogram showing the distribution of speed $v$ of all particles in one panel, and a histogram showing the distribution of kinetic energy, $E$ of all particles in the other (where $E=1/2 mv^2$). To be able to add the analytical formulae (the fitted functions) to these histograms, they should be probability density plots, meaning that the y-axis should be probability and the area under the curve of the histogram is equal to one.

**2b)** In two dimensions, the analytical expressions for the Maxwell-Boltzmann distribution of speed, $f(v)$, and the Boltzmann distribution, $g(E)$, are given by:

$$f(v)=\frac{mv}{k_B T}e^{-mv^2/2 k_B T},$$

$$g(E)=\frac{1}{k_B T}e^{-E/k_B T},$$

where $k_B = 1.38 \times 10^{-23} kg s^{-2} K^{-1}$ is the Boltzmann constant and $T$ is the temperature of the gas (a parameter that you need to determine). 

Your code should fit the analytical $f(v)$ to the $v$ histogram to find the temperature $T$ and plot this fitted function on top of the histogram. Although we can apply Poisson statistics to determine the uncertainties, the long tail in the Maxwell-Boltzmann distribution of speed and the relatively low number of total particles means that it would be very challenging to incorporate y-uncertainties into the fit in a way that will work _every_ time we run the simulation. As a result, you do not need to account for the uncertainties in the y-values when performing the fit.

Using the $T$ that you found from your fit of $f(v)$ to the $v$ histogram, plot the analytical $g(E)$ on top of the kinetic energy histogram.

**2c)** Although you do not need to incorporate y-value uncertainties into your fit, you still need to determine the uncertainty in the temperature that comes from the fit. As noted at https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html, the second returned array from this function, `pcov`, provides the estimated covariance of the first returned array `popt`: "The diagonals provide the variance of the parameter estimate. To compute one standard deviation errors (uncertainties) on the parameters use `perr = np.sqrt(np.diag(pcov))`."

You can communicate your temperature and its uncertainty in one of two ways.  The first option is that your code outputs a sentence giving the value of the temperature (and its uncertainty) found by fitting the speed distribution. The second, and slightly preferred option is that you include this information directly on the $v$ histogram. The displayed/printed uncertainty in $T$ should be rounded to two significant figures, and the displayed temperature rounded to have a matching number of decimal places to the rounded uncertainty. For example, if `mu = 13.443` and `dmu = 1.4533`, the displayed/printed value should read $$\mu = 13.4 \pm 1.5$$

## *Practical implementation*

**Step 1:** Start with the template program found in the mini-project 04 notebook, `project04.ipynb`, which prepares the gas particle distribution with following setup:

![](init.png)

Here the particles on the left side have an initial velocity of +500 m/s (to the right) and those on the right have an initial velocity of -500 m/s (to the left). You should colour particles initially found on the left-hand side blue and those initially found on the right-hand side red. Use the following parameters for the simulation (remember that it is best practice to implement these as variables within your code so that you can easily change them):

* Number of particles: 400
* Number of time steps: 1000 (500 animation frames)
* time step $\Delta t$ = 0.00002 s
* Particle size radius = 0.0015 m (much bigger than an actual air molecule)
* Particle mass = $2.672 \times 10^{-26} kg$ (this is the mass of an O2 molecule)
* Initial velocities: 500 m/s left or right, as indicated above.

Many of these parameters are already implemented in the template.

**Step 2:** Implement the particle-particle collision physics (velocity change) using the `itertool.combination()` method introduced above to identify which pairs of particles undergo collisions at each time step. The instruction to measure distances between particles is already in the program template, and you need to build the rest from these.

You may notice that the visual representation of the particle radius is much bigger than an actual molecule, but is still small enough that, given the step size and typical speeds, particles may pass through each other – that's ok. That value is chosen so you get a reasonable number of collisions in a reasonable amount of time.